In [ ]:
# Mount Google Drive untuk mengakses file
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Preprocessing Data Primer

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Word2vec/Dataset Labelled/Data Primer_Word2vec_Cosine Similarity/data_postingan_labeled.csv')
df.head(5)

,full_text,similarity_score,referensi_terdekat,label
0,Ini jadi bukti klaim2 gerakan dakwah ajaran ag...,0.878719,"Aktivitas Intoleransi, Radikalisme, dan Terori...",Radikal
1,@bossnya 5. Dakwah menyadarkan agar qt kembali...,0.847079,"Aktivitas Intoleransi, Radikalisme, dan Terori...",Radikal
2,@bnsphrxyzzz @mx00711 @agama_nusantara @sakkus...,0.782480,"Aktivitas intoleransi, radikalisme, dan terori...",Radikal
3,@Tita83079013 @MUIPusat @Kemenag_RI ISLAM ITU ...,0.786935,"Motif ideologi, politik, atau gangguan keamanan.",Radikal
4,Tradisi menarik lahir setelahnya: setiap Sulta...,0.738889,"Aktivitas intoleransi, radikalisme, dan terori...",Radikal


In [ ]:
df.label.value_counts()

,count
label,
Radikal,1018
Non-Radikal,468


Pre-processing Data : referensi https://github.com/adeariniputri/text-preprocesing

In [ ]:
pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 6.9 MB/s eta 0:00:00


In [ ]:
#Import library yang dibutuhkan
import csv
import nltk
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemover, ArrayDictionary
import string
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
#Proses filter
def filtering_text(text):
    # mengubah tweet menjadi huruf kecil
    text = text.lower()
    # menghilangkan url
    text = re.sub(r'https?:\/\/\S+','',text)
    # menghilangkan mention, link, hastag
    text = ' '.join(re.sub(r"([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", " ", text).split())
    #menghilangkan karakter byte (b')
    text = re.sub(r'(b\'{1,2})',"", text)
    # menghilangkan yang bukan huruf
    text = re.sub('[^a-zA-Z]', ' ', text)
    # menghilangkan digit angka
    text = re.sub(r'\d+', '', text)
    #menghilangkan tanda baca
    text = text.translate(str.maketrans("","",string.punctuation))
    # menghilangkan whitespace berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['filtered'] = df['full_text'].apply(filtering_text)
print(df['filtered'].head())

0    ini jadi bukti klaim gerakan dakwah ajaran aga...
1    dakwah menyadarkan agar qt kembali menerapkan ...
2    nusantara pedhet zelt transmigrasi jihadis isl...
3    ri islam itu bukan agama bro tetapi ideologi p...
4    tradisi menarik lahir setelahnya setiap sultan...
Name: filtered, dtype: object


In [ ]:
from tqdm import tqdm
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Proses stopwords dan stemming
def stop_stem(text):
    # Stopword
    with open('/content/drive/MyDrive/kamus.txt') as kamus:
        word = kamus.readlines()
        list_stopword = [line.replace('\n', "") for line in word]
    dictionary = ArrayDictionary(list_stopword)
    stopword = StopWordRemover(dictionary)
    text = stopword.remove(text)

    # Stemming
    factory_stemmer = StemmerFactory()
    stemmer = factory_stemmer.create_stemmer()
    text = stemmer.stem(text)
    return text

# Menambahkan progress bar dengan tqdm
tqdm.pandas()  # Ini akan memungkinkan penggunaan tqdm dengan apply()

# Menerapkan fungsi stop_stem dengan tqdm untuk progress bar
df['cleaned'] = df['filtered'].progress_apply(stop_stem)

# Menampilkan hasil
print(df['cleaned'].head())


100%|██████████| 1486/1486 [20:00<00:00,  1.24it/s]

0    jadi bukti klaim gera dakwah ajar agama para a...
1    dakwah sadar agar qt kembali terap sistem isla...
2    nusantara pedhet zelt transmigrasi jihad islam...
3    ri islam itu bukan agama bro tetapi ideologi p...
4    tradisi tarik lahir telah tiap sultan utsmaniy...
Name: cleaned, dtype: object


In [ ]:
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['tweet_tokens'] = df['cleaned'].apply(word_tokenize_wrapper)

print(df['tweet_tokens'].head())

0    [jadi, bukti, klaim, gera, dakwah, ajar, agama...
1    [dakwah, sadar, agar, qt, kembali, terap, sist...
2    [nusantara, pedhet, zelt, transmigrasi, jihad,...
3    [ri, islam, itu, bukan, agama, bro, tetapi, id...
4    [tradisi, tarik, lahir, telah, tiap, sultan, u...
Name: tweet_tokens, dtype: object


In [ ]:
import pandas as pd

# Membaca kamus normalisasi dari file CSV
kamus_normalisasi = pd.read_csv("/content/drive/MyDrive/slang.csv", header=None)

# Membuat dictionary untuk kamus normalisasi
kata_normalisasi_dict = {}

# Mengisi dictionary dengan data dari kamus normalisasi
for index, row in kamus_normalisasi.iterrows():
    # Pastikan menggunakan iloc untuk akses yang benar pada posisi indeks
    kata_normalisasi_dict[row.iloc[0]] = row.iloc[1]

# Fungsi untuk normalisasi kata
def normalisasi_kata(document):
    # Normalisasi setiap kata dalam dokumen jika ada di kamus
    return [kata_normalisasi_dict[term] if term in kata_normalisasi_dict else term for term in document]

# Terapkan normalisasi ke kolom 'tweet_tokens'
df['normalisasi'] = df['tweet_tokens'].apply(normalisasi_kata)

# Menggabungkan token dalam kolom 'normalisasi' menjadi kalimat utuh
df['normalisasi_kalimat'] = df['normalisasi'].apply(lambda tokens: ' '.join(tokens))

# Menampilkan hasil
print(df[['normalisasi_kalimat']].head(10))


                                 normalisasi_kalimat
0  jadi bukti klaim gera dakwah ajar agama para a...
1  dakwah sadar agar qt kembali terap sistem isla...
2  nusantara pedhet zelt transmigrasi jihad islam...
3  ri islam itu bukan agama bro tetapi ideologi p...
4  tradisi tarik lahir telah tiap sultan utsmaniy...
5  holland nada gaming muda grey lawan nada follo...
6  nasehat multaqa ulama aswaja jatim kepada umat...
7  hijrah hti akan dakwah sampai khilafah tegak n...
8  karena bahas palestina khilafah jadi hti sampa...
9  klaim pentol hti paling jujur ngawur minimal j...


In [ ]:
# Menyimpan hasil Pre-processing Data

df.to_csv("/content/drive/MyDrive/Word2vec/Dataset Labelled/Data Primer_Word2vec_Cosine Similarity/data_prepocessing.csv", index=False)

# Preprocessing Data Primer + Data Augmentasi

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Word2vec/Dataset Labelled/Data Primer_Data Augmentasi_Word2vec_Cosine Similarity/data_postingan_labeled_augment.csv')
df.head(5)

,full_text,similarity_score,referensi_terdekat,label
0,Ini jadi bukti klaim2 gerakan dakwah ajaran ag...,0.878719,"Aktivitas Intoleransi, Radikalisme, dan Terori...",Radikal
1,@bossnya 5. Dakwah menyadarkan agar qt kembali...,0.847079,"Aktivitas Intoleransi, Radikalisme, dan Terori...",Radikal
2,@bnsphrxyzzz @mx00711 @agama_nusantara @sakkus...,0.782480,"Aktivitas intoleransi, radikalisme, dan terori...",Radikal
3,@Tita83079013 @MUIPusat @Kemenag_RI ISLAM ITU ...,0.786935,"Motif ideologi, politik, atau gangguan keamanan.",Radikal
4,Tradisi menarik lahir setelahnya: setiap Sulta...,0.738889,"Aktivitas intoleransi, radikalisme, dan terori...",Radikal


In [ ]:
df.label.value_counts()

,count
label,
Radikal,1400
Non-Radikal,1400


Pre-processing Data : referensi https://github.com/adeariniputri/text-preprocesing

In [ ]:
#Import library yang dibutuhkan
import csv
import nltk
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemover, ArrayDictionary
import string
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
#Proses filter
def filtering_text(text):
    # mengubah tweet menjadi huruf kecil
    text = text.lower()
    # menghilangkan url
    text = re.sub(r'https?:\/\/\S+','',text)
    # menghilangkan mention, link, hastag
    text = ' '.join(re.sub(r"([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", " ", text).split())
    #menghilangkan karakter byte (b')
    text = re.sub(r'(b\'{1,2})',"", text)
    # menghilangkan yang bukan huruf
    text = re.sub('[^a-zA-Z]', ' ', text)
    # menghilangkan digit angka
    text = re.sub(r'\d+', '', text)
    #menghilangkan tanda baca
    text = text.translate(str.maketrans("","",string.punctuation))
    # menghilangkan whitespace berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['filtered'] = df['full_text'].apply(filtering_text)
print(df['filtered'].head())

0    ini jadi bukti klaim gerakan dakwah ajaran aga...
1    dakwah menyadarkan agar qt kembali menerapkan ...
2    nusantara pedhet zelt transmigrasi jihadis isl...
3    ri islam itu bukan agama bro tetapi ideologi p...
4    tradisi menarik lahir setelahnya setiap sultan...
Name: filtered, dtype: object


In [ ]:
from tqdm import tqdm
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Proses stopwords dan stemming
def stop_stem(text):
    # Stopword
    with open('/content/drive/MyDrive/kamus.txt') as kamus:
        word = kamus.readlines()
        list_stopword = [line.replace('\n', "") for line in word]
    dictionary = ArrayDictionary(list_stopword)
    stopword = StopWordRemover(dictionary)
    text = stopword.remove(text)

    # Stemming
    factory_stemmer = StemmerFactory()
    stemmer = factory_stemmer.create_stemmer()
    text = stemmer.stem(text)
    return text

# Menambahkan progress bar dengan tqdm
tqdm.pandas()  # Ini akan memungkinkan penggunaan tqdm dengan apply()

# Menerapkan fungsi stop_stem dengan tqdm untuk progress bar
df['cleaned'] = df['filtered'].progress_apply(stop_stem)

# Menampilkan hasil
print(df['cleaned'].head())


100%|██████████| 2800/2800 [27:53<00:00,  1.67it/s]

0    jadi bukti klaim gera dakwah ajar agama para a...
1    dakwah sadar agar qt kembali terap sistem isla...
2    nusantara pedhet zelt transmigrasi jihad islam...
3    ri islam itu bukan agama bro tetapi ideologi p...
4    tradisi tarik lahir telah tiap sultan utsmaniy...
Name: cleaned, dtype: object


In [ ]:
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['tweet_tokens'] = df['cleaned'].apply(word_tokenize_wrapper)

print(df['tweet_tokens'].head())

0    [jadi, bukti, klaim, gera, dakwah, ajar, agama...
1    [dakwah, sadar, agar, qt, kembali, terap, sist...
2    [nusantara, pedhet, zelt, transmigrasi, jihad,...
3    [ri, islam, itu, bukan, agama, bro, tetapi, id...
4    [tradisi, tarik, lahir, telah, tiap, sultan, u...
Name: tweet_tokens, dtype: object


In [ ]:
import pandas as pd

# Membaca kamus normalisasi dari file CSV
kamus_normalisasi = pd.read_csv("/content/drive/MyDrive/slang.csv", header=None)

# Membuat dictionary untuk kamus normalisasi
kata_normalisasi_dict = {}

# Mengisi dictionary dengan data dari kamus normalisasi
for index, row in kamus_normalisasi.iterrows():
    # Pastikan menggunakan iloc untuk akses yang benar pada posisi indeks
    kata_normalisasi_dict[row.iloc[0]] = row.iloc[1]

# Fungsi untuk normalisasi kata
def normalisasi_kata(document):
    # Normalisasi setiap kata dalam dokumen jika ada di kamus
    return [kata_normalisasi_dict[term] if term in kata_normalisasi_dict else term for term in document]

# Terapkan normalisasi ke kolom 'tweet_tokens'
df['normalisasi'] = df['tweet_tokens'].apply(normalisasi_kata)

# Menggabungkan token dalam kolom 'normalisasi' menjadi kalimat utuh
df['normalisasi_kalimat'] = df['normalisasi'].apply(lambda tokens: ' '.join(tokens))

# Menampilkan hasil
print(df[['normalisasi_kalimat']].head(10))


                                 normalisasi_kalimat
0  jadi bukti klaim gera dakwah ajar agama para a...
1  dakwah sadar agar qt kembali terap sistem isla...
2  nusantara pedhet zelt transmigrasi jihad islam...
3  ri islam itu bukan agama bro tetapi ideologi p...
4  tradisi tarik lahir telah tiap sultan utsmaniy...
5  holland nada gaming muda grey lawan nada follo...
6  nasehat multaqa ulama aswaja jatim kepada umat...
7  hijrah hti akan dakwah sampai khilafah tegak n...
8  karena bahas palestina khilafah jadi hti sampa...
9  klaim pentol hti paling jujur ngawur minimal j...


In [ ]:
# Menyimpan hasil Pre-processing Data

df.to_csv("/content/drive/MyDrive/Word2vec/Dataset Labelled/Data Primer_Data Augmentasi_Word2vec_Cosine Similarity/data_prepocessing_augmentasi.csv", index=False)